In [ ]:
#Библиотеки для работы с REST-запросами на сайт hh.ru
import pickle, requests, json
url = "https://api.hh.ru/areas/"
payload={}
headers = {
  'Cookie': '__ddg1=QKQYID8KtfzzXrx8bXqg; __ddgid=dxh5TtpATVgtTQIx; __ddgmark=rPcNyv43PstBDlCg'
}

#Выполнение запроса на получение списка городов и их аргументов
response = requests.request("GET", url, headers=headers, data=payload)
b = response.json()
citiesID = {}


#Получение уникального id города в пределах РФ
for region in b[0]['areas']:
    for city in region['areas']:
        #print(city)
        citiesID[city['name']] = int(city['id'])
        #break
   # break

#Присваивание городам из выборки id для последущего REST-запроса
with open('./New folder/d1.pickle','rb') as my:
    d = pickle.load(my)  
    for city in d.keys():
        if city == 'Москва': #Для ускорения присваивания id
            d[city][1]= 1
        elif city == 'Санкт-Петербург': #Для ускорения присваивания id
            d[city][1]= 2
        elif city == 'Иваново': #В базе HH есть несколько населёных пунктов с таким названием
            d[city][1]= 32
        elif city == 'Новоросийск': #Опечатка в выборке
            d[city][1]= 1454
        else:
            #Поиск id из результата запроса
            for restCities in citiesID.keys():
                if restCities == city:
                    d[city][1]= citiesID[city]
                    break
                d[city][1]= 1

In [ ]:
#Библиотеки для работы с REST-запросами на сайт hh.ru
import requests,json,time,os,pandas as pd

    
def getPage(page = 0, empText='Аналитик', empArea=1):
    """
    Создаем метод для получения страницы со списком вакансий.
    Аргументы:
        page - Индекс страницы, начинается с 0. Значение по умолчанию 0, т.е. первая страница
    """
     
    # Справочник для параметров GET-запроса
    params = {
        'text': 'NAME:'+empText, # Текст фильтра. В имени должно быть слово "Аналитик"
        'area': empArea, # Поиск ощуществляется по вакансиям города Москва
        'page': page, # Индекс страницы поиска на HH
        'per_page': 10 # Кол-во вакансий на 1 странице
    }
     
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    data = req.content.decode() # Декодируем его ответ, чтобы Кириллица отображалась корректно
    req.close()
    return data
 
def mainProcess(x,y,z):# Считываем первые 2000 вакансий
    for page in range(0, 1):
        print('./New folder/emp/'+str(z)+'/'+str(x))

        # Преобразуем текст ответа запроса в справочник Python
        jsObj = json.loads(getPage(page,x,y))

        # Сохраняем файлы в папку {путь до текущего документа со скриптом}\docs\pagination
        # Определяем количество файлов в папке для сохранения документа с ответом запроса
        # Полученное значение используем для формирования имени документа
        nextFileName = './New folder/preprod/{}.json'.format(len(os.listdir('./New folder/preprod')))

        # Создаем новый документ, записываем в него ответ запроса, после закрываем
        f = open(nextFileName, mode='w', encoding='utf8')
        f.write(json.dumps(jsObj, ensure_ascii=False))
        f.close()

        # Проверка на последнюю страницу, если вакансий меньше 2000
        if (jsObj['pages'] - page) <= 1:
            break

        # Необязательная задержка, но чтобы не нагружать сервисы hh, оставим. 5 сек мы может подождать
        time.sleep(5)

    #print('Старницы поиска собраны')
    # Получаем перечень ранее созданных файлов со списком вакансий и проходимся по нему в цикле 
    
    if os.path.exists('./New folder/emp/'+str(z)+'/'+str(x)) == False:
        if os.path.exists('./New folder/emp/'+str(z)) == False:
            os.mkdir('./New folder/emp/'+str(z))
        os.mkdir('./New folder/emp/'+str(z)+'/'+str(x))
        
    for fl in os.listdir('./New folder/preprod/'):

        # Открываем файл, читаем его содержимое, закрываем файл
        f = open('./New folder/preprod/{}'.format(fl), encoding='utf8')
        jsonText = f.read()
        f.close()

        # Преобразуем полученный текст в объект справочника
        jsonObj = json.loads(jsonText)

        # Получаем и проходимся по непосредственно списку вакансий
        for v in jsonObj['items']:

            # Обращаемся к API и получаем детальную информацию по конкретной вакансии
            req = requests.get(v['url'])
            data = req.content.decode()
            req.close()

            # Создаем файл в формате json с идентификатором вакансии в качестве названия
            # Записываем в него ответ запроса и закрываем файл
            fileName = './New folder/emp/'+str(z)+'/'+str(x)+'/{}.json'.format(v['id'])
            f = open(fileName, mode='w', encoding='utf8')
            #data.to_csv()
            f.write(data)
            f.close()
            

            time.sleep(0.25)
    for file in os.listdir('./New folder/preprod/'):
        os.remove('./New folder/preprod/'+file)


In [ ]:
#Вызов основных функций алгоритма
for city in d.keys():
    for empVacancy in d[city][0]:
        #getPage(0,empVacancy,d[city][1])
        mainProcess(empVacancy.strip(),d[city][1],city)